In [1]:
import torchaudio
import torch 
import pandas as pd
import numpy as np
import re 
import os 
from torch.nn import functional as F
import random
import math
import pickle
import gc
import torch.nn as nn
from tqdm.notebook import tqdm
import os
import glob
import warnings
from torch.utils.data import Dataset, DataLoader
import glob
warnings.filterwarnings(action='ignore')

In [2]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print(DEVICE)

cuda


In [19]:
def make_min2_mel(song_paths='/home/work/Tcae_apply/tcae_inference_folder/wav_folder/존박_빗속에서.wav',sample_rate=16000): # song_paths = wav_path 입력
    
    wav, wav_sample_rate = torchaudio.load(song_paths)

    if wav_sample_rate != sample_rate:
        transform = torchaudio.transforms.Resample(wav_sample_rate, sample_rate)
        wav = transform(wav)


    wav = torch.unsqueeze(torch.mean(wav, axis=0), dim=0) # mono
    wav_to_mel = torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_fft=512, hop_length=256, n_mels=48) # to mel

    mel = wav_to_mel(wav)
    mel = mel.cpu().detach().numpy() # 멜 임베딩을 numpy로

    return mel # return : mel_embedding
    
        #song_name = os.path.splitext(song_paths[j])[0].split('/')[-1]
        #np.save(f"{save_dir}/{song_name}.npy", mel) # 멜 스펙토그램_embedding이 저장되는 경로

In [ ]:
'/home/work/Tcae_apply/model_dir3/Tcae_apply/tcae_inference_folder/wav_folder/정하_보컬.npy'

In [72]:
a = np.load('/home/work/Tcae_apply/tcae_inference_folder/mel_folder/성현_보컬.npy')

In [73]:
a.shape

(1, 48, 8099)

In [20]:
#charlie
############# 원래 ##############
class TimeAutoEncoder(nn.Module):
    def __init__(self):
        super(TimeAutoEncoder, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channels = 48, out_channels = 512, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )
    
        self.conv2 = nn.Sequential(
            nn.Conv1d(in_channels = 512, out_channels = 256, kernel_size = 3, stride = 1, padding = 0, dilation = 2),
            #nn.BatchNorm1d(256),
            nn.ReLU(),
        )
            
        self.conv3 = nn.Sequential(
            nn.Conv1d(in_channels = 256, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 4),
            #nn.BatchNorm1d(128),
            nn.ReLU(),
        )

        self.conv4 = nn.Sequential(
            nn.Conv1d(in_channels = 128, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 8),
            #nn.BatchNorm1d(64),
            nn.ReLU(),
        )

        self.conv5 = nn.Sequential(
            nn.Conv1d(in_channels = 64, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 16),
            #nn.BatchNorm1d(32),
            nn.ReLU(),
        )

        self.conv6 = nn.Sequential(
            nn.Conv1d(in_channels = 32, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 32),
            #nn.BatchNorm1d(16),
            nn.ReLU(),
        )

        self.conv7 = nn.Sequential(
            nn.Conv1d(in_channels = 16, out_channels = 4, kernel_size = 3, stride = 1, padding = 0, dilation = 64),
            #nn.BatchNorm1d(8),
            nn.ReLU(),
        )

        self.encoder_fc = nn.Sequential(
            nn.Linear(8 * 7501, 256),
            #nn.BatchNorm1d(128),
            nn.LeakyReLU(),
        )
        
        self.decoder_fc = nn.Sequential(
            nn.Linear(256, 8 * 7501),
            nn.ReLU(),
        )

        self.t_conv1 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 8, out_channels = 16, kernel_size  = 3, stride = 1, dilation=62),
            nn.Conv1d(in_channels = 16, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            #nn.BatchNorm1d(16),
            nn.ReLU(),
        )

        self.t_conv2 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 16, out_channels = 32, kernel_size  = 3, stride = 1, dilation = 30),
            nn.Conv1d(in_channels = 32, out_channels = 32, kernel_size = 3, stride = 1, padding = 0, dilation = 2),
            #nn.BatchNorm1d(32),
            nn.ReLU(),
        )

        self.t_conv3 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 32, out_channels = 64, kernel_size  = 3, stride = 1, dilation=14),
            nn.Conv1d(in_channels = 64, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 4),
            #nn.BatchNorm1d(64),
            nn.ReLU(),
        )

        self.t_conv4 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 64, out_channels = 128, kernel_size  = 3, stride = 1, dilation = 6),
            nn.Conv1d(in_channels = 128, out_channels = 128, kernel_size = 3, stride = 1, padding = 0, dilation = 8),
            #nn.BatchNorm1d(128),
            nn.ReLU(),
        )

        self.t_conv5 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 128, out_channels = 256, kernel_size  = 3, stride = 1, dilation=2),
            nn.Conv1d(in_channels = 256, out_channels = 256, kernel_size = 3, stride = 1, padding = 0, dilation = 16),
            #nn.BatchNorm1d(256),
            nn.ReLU(),
        )

        self.t_conv6 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 256, out_channels = 512, kernel_size  = 3, stride = 1, dilation = 1),
            nn.Conv1d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 0, dilation = 32),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )

        self.t_conv7 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 512, out_channels = 48, kernel_size  = 3, stride = 1, dilation= 1),
            nn.Conv1d(in_channels = 1024, out_channels = 48, kernel_size = 3, stride = 1, padding = 0, dilation = 64)
        )

        
        
        
        self.conv1to3 = nn.Sequential(
            nn.Conv1d(in_channels = 512, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 4),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )# out : 128 * 7501
        
        self.conv2to4 = nn.Sequential(
            nn.Conv1d(in_channels = 256, out_channels = 32, kernel_size = 3, stride = 1, padding = 0, dilation = 8),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )# out : 64 * 7501
        
        self.conv3to5 = nn.Sequential(
            nn.Conv1d(in_channels = 128, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 16),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )# out : 32 * 7501
        
        self.conv4to6 = nn.Sequential(
            nn.Conv1d(in_channels = 64, out_channels = 8, kernel_size = 3, stride = 1, padding = 0, dilation = 32),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )# out : 16 * 7501
        
        self.conv5to7 = nn.Sequential(
            nn.Conv1d(in_channels = 32, out_channels = 4, kernel_size = 3, stride = 1, padding = 0, dilation = 64),
            #nn.BatchNorm1d(512),
            nn.ReLU(),
        )# out : 8 * 7501
        
        
        
    def forward(self, mel_spec):
        x = F.pad(mel_spec, pad = (2, 0, 0, 0))
        x1 = self.conv1(x) # 48 * 7501 => 512 * 7501 
        print(x1.shape)
        
        x1to2 = F.pad(x1, pad = (4, 0, 0, 0)) #512 * 7501 
        x1to3 = F.pad(x1, pad = (8, 0, 0, 0)) #512 * 7501 
        #print('x1to2',x1to2.shape)
        #print('x1to3',x1to3.shape)
        
        
        x2 = self.conv2(x1to2) # 512 * 7501 => 256 * 7501
        print(x2.shape)
        
        x2to3 = F.pad(x2, pad = (8, 0, 0, 0)) # 256 * 7501
        #x2to4 = F.pad(x2, pad = (16, 0, 0, 0)) # 256 * 7501
        #print('x2to3 :',x2to3.shape)
        #print('x2to4 :',x2to3.shape)
        
        x3 = self.conv3(x2to3) # 256 * 7501 => 64 * 7501
        x3_connec = self.conv1to3(x1to3) # In : 512 * 7501 , out: 64 * 7501     128
        x3= torch.cat([x3,x3_connec],1) # 128 * 7501
        print(x3.shape)
        
        
        x3to4 = F.pad(x3, pad = (16, 0, 0, 0)) # 128 * 7501
        x3to5 = F.pad(x3, pad = (32, 0, 0, 0)) # 128 * 7501
        #print('x3to4 :',x3to4.shape)
        #print('x3to5 :',x3to5.shape)
        
        x4 = self.conv4(x3to4) # 128 * 7501 => 64 * 7501
        #x4_connec = self.conv2to4(x2to4) # In : 256 * 7501 , out : 32 * 7501 
        #x4 = torch.cat([x4,x4_connec],1) # 64 * 7501
        print(x4.shape)
        
        x4to5 = F.pad(x4, pad = (32, 0, 0, 0)) # 64 * 7501
        #x4to6 = F.pad(x4, pad = (64, 0, 0, 0)) # 64 * 7501
        #print('x4to5 :',x4to5.shape)
        #print('x4to6 :',x4to6.shape)
        
        
        x5 = self.conv5(x4to5) # 64 * 7501 = > 16 * 7501 
        x5_connec = self.conv3to5(x3to5) # 128 * 7501 => 16 * 7501
        x5 = torch.cat([x5,x5_connec],1) # 32 * 7501
        print(x5.shape)
        
        x5to6 = F.pad(x5, pad = (64, 0, 0, 0)) # 32 * 7501
        x5to7 = F.pad(x5, pad = (128, 0, 0, 0)) # 32 * 7501
        #print('x5to6 :',x5to6.shape)
        #print('x5to7 :',x5to7.shape)
        
        
        x6 = self.conv6(x5to6) # 32 * 7501 => 16 * 7501
        #x6_connec = self.conv4to6(x4to6) # 64 * 7501 => 8 * 7501
        #x6 = torch.cat([x6,x6_connec],1) # 16 * 7501
        print(x6.shape)
        x6to7 = F.pad(x6, pad = (128, 0, 0, 0)) # 32 * 7501
        #print('x6to7 :',x6to7.shape)
        
        
        x7 = self.conv7(x6to7) # 16 * 7501 => 4 *7501 
        pre_encode = torch.flatten(x7)
        x7_connec = self.conv5to7(x5to7) # 32 * 7501 => 4 * 7501
        x7 = torch.cat([x7,x7_connec],1) # 8 * 7501
        print(x7.shape)
        encode = self.encoder_fc(x7.view(-1, 8 * 7501))

        #encode = self.encoder_fc(x.view(-1, 8 * 1876))

        # print('decode')
        x = self.decoder_fc(encode)
        x = x.view(-1, 8, 7501)
        
        #x = torch.swapaxes(torch.fliplr(torch.swapaxes(x, 1, 2)), 1, 2)
        
        x = torch.cat([x7,x],1) # 4,16,7501
        #x = self.concat7_shape(x) 
        x = F.pad(x, pad = (2, 0, 0, 0)) 
        x = self.t_conv1(x) # 4,8,7501
        #print(x.shape,'new_cat7')
        
        
        x = torch.cat([x6,x],1)
        #x = self.concat6_shape(x)
        x = F.pad(x, pad = (4, 0, 0, 0))
        x = self.t_conv2(x)
        
        
        x = torch.cat([x5,x],1)
        #x = self.concat5_shape(x)
        x = F.pad(x, pad = (8, 0, 0, 0))
        x = self.t_conv3(x)
        
        
        
        x = torch.cat([x4,x],1)
        #x = self.concat4_shape(x)
        x = F.pad(x, pad = (16, 0, 0, 0))
        x = self.t_conv4(x)
        
        
        x = torch.cat([x3,x],1)
        #x = self.concat3_shape(x)
        x = F.pad(x, pad = (32, 0, 0, 0))
        x = self.t_conv5(x)
        
        
        
        x = torch.cat([x2,x],1)
        #x = self.concat2_shape(x)
        x = F.pad(x, pad = (64, 0, 0, 0))
        x = self.t_conv6(x)
        
        
        
        x = torch.cat([x1,x],1)
        #x = self.concat1_shape(x)
        x = F.pad(x, pad = (128, 0, 0, 0))
        x = self.t_conv7(x)
        
        #print(x.shape)
        #x = torch.swapaxes(torch.fliplr(torch.swapaxes(x, 1, 2)), 1, 2)
        
        return encode, x


In [21]:
def get_mel_embeding(model, train_loader):
    model.eval()
    mel_embeding_li = []
    with torch.no_grad():
        for batch in train_loader: 
            
            mel = torch.FloatTensor(batch).to(DEVICE)
            
            encode, output = model(mel)
            mel_embeding_li.append(encode.detach().cpu().numpy())

    return mel_embeding_li

In [22]:
def mel_to_embedding():
    mel = make_min2_mel(song_paths = '/home/work/Tcae_apply/tcae_inference_folder/wav_folder/존박_빗속에서.wav')
    mel = [mel.squeeze()[:,:7501]]
    inference_batch_li = DataLoader(mel, batch_size=1, shuffle=False,drop_last=False)

    inference_model = TimeAutoEncoder().to(DEVICE)
    inference_model.load_state_dict(torch.load('/home/work/Tcae_apply/model_dir3/TimeAutoEncoder_skipconnection_charlie_val.pt', map_location = DEVICE))

    inference_embedding = get_mel_embeding(inference_model,inference_batch_li)
    return inference_embedding
    #pd.DataFrame(inference_embedding[0]).to_csv('TJAE/tjae_vocal.csv')

In [23]:
mel_to_embedding()

torch.Size([1, 512, 7501])
torch.Size([1, 256, 7501])
torch.Size([1, 128, 7501])
torch.Size([1, 64, 7501])
torch.Size([1, 32, 7501])
torch.Size([1, 16, 7501])
torch.Size([1, 8, 7501])


[array([[-9.30863142e-01, -9.04240012e-01, -7.08037674e-01,
         -9.41487491e-01, -9.50743794e-01, -9.85211015e-01,
         -8.91448498e-01, -9.31205511e-01, -9.38048005e-01,
         -8.72863829e-01, -8.37759018e-01, -8.56580257e-01,
         -9.28934693e-01, -1.00395739e+00, -1.08263075e+00,
         -8.01653862e-01, -9.14770961e-01, -8.90620351e-01,
         -8.71596038e-01, -1.09121323e+00, -9.62611377e-01,
         -8.80256712e-01, -1.04625058e+00, -9.40831482e-01,
         -1.06616533e+00, -9.47073817e-01, -7.38908589e-01,
         -2.02625847e+00, -1.02972150e+00, -1.03506660e+00,
         -8.12007725e-01, -1.06731236e+00, -7.95435369e-01,
         -1.08916581e+00, -9.67356980e-01, -9.11371887e-01,
         -1.05690873e+00, -8.90077174e-01, -1.06327176e+00,
         -8.43641818e-01, -1.03541291e+00, -8.35502088e-01,
         -8.28084290e-01, -8.74129534e-01, -1.05727637e+00,
         -1.09753442e+00, -1.00423384e+00, -1.08074403e+00,
         -9.72967327e-01, -8.58922660e-0

In [44]:
base_dir = '/home/work/'
inference_wav_path = base_dir + '/Tcae_apply/tcae_inference_folder/wav_folder/' # inference_wav가 '저장된 경로' 
#inference_mel_path = base_dir + '/Tcae_apply/tcae_inference_folder/mel_folder/' # inference_mel이 저장될 경로

inference = [inference_wav_path+x for x in os.listdir(inference_wav_path)] # inference_wav list

mel = make_min2_mel(song_paths = inference)
mel = [mel.squeeze()[:,:7501]]
inference_batch_li = DataLoader(mel, batch_size=1, shuffle=False,drop_last=False)
# mel_embedding이 저장되는 경로
# return inference의 멜 스펙토그램 npy

  0%|          | 0/2 [00:00<?, ?it/s]

In [45]:
inference_model = TimeAutoEncoder().to(DEVICE)
model_dir = '/home/work/Tcae_apply/model_dir3/'
inference_model.load_state_dict(torch.load(model_dir + f'TimeAutoEncoder_skipconnection_charlie_val.pt', map_location = DEVICE))

<All keys matched successfully>

In [47]:
inference_embedding = get_mel_embeding(inference_model,inference_batch_li)

In [ ]:
'/home/work/Tcae_apply/model_dir3/TimeAutoEncoder_skipconnection_charlie_val.pt'

In [61]:
import pandas as pd

In [67]:
pd.DataFrame(inference_embedding[0])

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,-11.367964,-11.209202,-5.204,-11.691737,-12.434955,-11.375682,-3.83262,-11.711182,-11.474288,-10.857143,...,-12.475056,-13.220174,-12.013934,328.706451,-11.837992,-12.600722,-12.358846,-13.43066,-12.939454,-18.133314


In [ ]:
def make_inference_embedding():
    inference_model = TimeAutoEncoder().to(DEVICE)
    model_dir = '/home/work/Tcae_apply/model_dir3/'
    inference_model.load_state_dict(torch.load(model_dir + f'TimeAutoEncoder_skipconnection_charlie_val.pt', map_location = DEVICE))

In [ ]:
#model = TimeAutoEncoder().to(DEVICE)

#model_dir = '/home/work/Tcae_apply/model_dir3/'
#model.load_state_dict(torch.load(model_dir + f'TimeAutoEncoder_skipconnection_charlie_val.pt', map_location = DEVICE))

In [ ]:
'/home/work/Tcae_apply/tcae_inference_folder/wav_folder/정하_보컬.wav'

In [76]:
song_path = ['/home/work/Tcae_apply/tcae_inference_folder/wav_folder/정하_보컬.wav']
mel = make_min2_mel(song_paths = song_path)# 멜 스펙토그램 npy 


  0%|          | 0/1 [00:00<?, ?it/s]

In [80]:
mel.squeeze()[:,:7501].shape

(48, 7501)

In [83]:
a.shape

(1, 48, 8099)

In [99]:
b = [[[1,2,3]],[[2,4,5]]]

In [ ]:
(48,7501)

In [105]:
np.concatenate(b)

array([[1, 2, 3],
       [2, 4, 5]])

In [84]:
inference_npy_path = '/home/work/Tcae_apply/tcae_inference_folder/mel_folder/'
inference_npy_path = glob.glob(inference_npy_path + '*.npy')
inference_list = [np.load(song_path).squeeze()[:,:7501] for song_path in tqdm(inference_npy_path)]
inference_batch_li = DataLoader(inference_list, batch_size=1, shuffle=False,drop_last=False)

  0%|          | 0/3 [00:00<?, ?it/s]

In [95]:
inference_embedding_li = get_mel_embeding(model = inference_model, train_loader = inference_batch_li)
inference_embedding = np.concatenate(inference_embedding_li, axis = 0)

In [125]:
b = np.load('/home/work/Tcae_apply/tcae_inference_folder/mel_folder/정하_보컬.npy')

In [132]:
b.shape

(1, 48, 13233)

In [133]:
c = b.squeeze()[:,:7501]

In [134]:
c.shape

(48, 7501)

In [135]:
inference_batch_li = DataLoader([c], batch_size=1, shuffle=False,drop_last=False)

In [136]:
inference_embedding_li = get_mel_embeding(model = inference_model, train_loader = inference_batch_li)

In [ ]:
[(48,7501)]

In [ ]:
[(48,7501),(48,7501),(48,7501)]

In [97]:
inference_embedding.shape

(3, 256)

In [98]:
a.shape

(1, 48, 8099)